## Final Project : IPO prediction (classification)

In [2]:
import sklearn 
import random
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, make_scorer, confusion_matrix, accuracy_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv("data_final.csv")
df.head()
# print(df.columns)

,Unnamed: 0,ipo,name,funded_object_id,num_investors,raised_amount_usd,type_Bachelor,type_Master,type_Other,type_OtherDiploma,...,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,state_not specified
0,0,0,Wetpaint,c:1,9,39750000.0,7.0,5.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,0,FriendFeed,c:1001,3,5000000.0,4.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,Mobclix,c:10014,1,0.0,2.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,Fitbit,c:10015,19,68069200.0,2.0,7.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,MTPV,c:100155,5,10125293.0,1.0,3.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df.drop(columns=['Unnamed: 0'], inplace=True)
df

,ipo,name,funded_object_id,num_investors,raised_amount_usd,type_Bachelor,type_Master,type_Other,type_OtherDiploma,type_PhD,...,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,state_not specified
0,0,Wetpaint,c:1,9,39750000.0,7.0,5.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0,FriendFeed,c:1001,3,5000000.0,4.0,6.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,Mobclix,c:10014,1,0.0,2.0,1.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,Fitbit,c:10015,19,68069200.0,2.0,7.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,MTPV,c:100155,5,10125293.0,1.0,3.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9302,0,DECA,c:990,7,15000000.0,2.0,3.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9303,0,wunderloop,c:992,4,10000000.0,1.0,4.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9304,0,FastCustomer,c:99669,6,750000.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9305,0,Zimbra,c:997,9,14500000.0,2.0,7.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_w_dup = df.append(df[df['ipo'] == 1] * 20, ignore_index=True)

In [6]:
np.sum(df_w_dup['ipo'])

4830

In [7]:
features = df_w_dup.columns
features = features.drop(['name', 'ipo','funded_object_id'])

In [8]:
#scaling on features
df_w_dup[features] = StandardScaler().fit_transform(df_w_dup[features])
df_w_dup[features]

,num_investors,raised_amount_usd,type_Bachelor,type_Master,type_Other,type_OtherDiploma,type_PhD,subject_Accounting,subject_Accounting and Finance,subject_Advertising,...,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,state_not specified
0,-0.000104,-0.041682,0.005632,-0.027913,-0.020992,-0.029696,-0.045333,-0.103101,-0.045438,-0.036035,...,-0.029621,-0.075032,-0.042832,-0.0594,-0.035494,0.920750,-0.02603,-0.014483,-0.014483,-0.044961
1,-0.180838,-0.062679,-0.024166,-0.019331,-0.060929,-0.029696,-0.045333,-0.103101,-0.045438,-0.036035,...,-0.029621,-0.075032,-0.042832,-0.0594,-0.035494,-0.076433,-0.02603,-0.014483,-0.014483,-0.044961
2,-0.241083,-0.065700,-0.044031,-0.062242,0.018944,-0.029696,-0.045333,-0.103101,-0.045438,-0.036035,...,-0.029621,-0.075032,-0.042832,-0.0594,-0.035494,-0.076433,-0.02603,-0.014483,-0.014483,-0.044961
3,0.301119,-0.024570,-0.044031,-0.010749,0.018944,-0.029696,-0.082749,-0.103101,-0.045438,-0.036035,...,-0.029621,-0.075032,-0.042832,-0.0594,-0.035494,-0.076433,-0.02603,-0.014483,-0.014483,-0.044961
4,-0.120594,-0.059582,-0.053964,-0.045078,-0.060929,-0.029696,-0.045333,-0.103101,-0.045438,-0.036035,...,-0.029621,0.924379,-0.042832,-0.0594,-0.035494,-0.076433,-0.02603,-0.014483,-0.014483,-0.044961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9532,0.933689,0.085359,0.333410,0.100819,1.536531,-0.029696,-0.082749,-0.103101,-0.045438,-0.036035,...,-0.029621,-0.075032,-0.042832,-0.0594,-0.035494,-0.076433,-0.02603,-0.014483,-0.014483,-0.044961
9533,0.331242,0.175994,-0.063896,0.272462,-0.060929,-0.029696,2.162214,-0.103101,-0.045438,-0.036035,...,-0.029621,-0.075032,-0.042832,-0.0594,-0.035494,-0.076433,-0.02603,-0.014483,-0.014483,-0.044961
9534,9.367949,0.466027,1.723981,2.503821,0.737801,-0.029696,1.413893,7.317825,-0.045438,-0.036035,...,-0.029621,-0.075032,-0.042832,-0.0594,-0.035494,-0.076433,-0.02603,-0.014483,-0.014483,-0.044961
9535,2.138583,1.102021,2.319941,0.787391,-0.060929,-0.029696,2.910535,-0.103101,-0.045438,-0.036035,...,-0.029621,-0.075032,-0.042832,-0.0594,-0.035494,19.867216,-0.02603,-0.014483,-0.014483,-0.044961


In [9]:
# computing weighted accuracy from CS4780
def weighted_accuracy(pred, true):
    assert(len(pred) == len(true))
    num_labels = len(true)
    num_pos = sum(true)
    num_neg = num_labels - num_pos
    frac_pos = num_pos/num_labels
    weight_pos = 1/frac_pos
    weight_neg = 1/(1-frac_pos)
    num_pos_correct = 0
    num_neg_correct = 0
    for pred_i, true_i in zip(pred, true):
        num_pos_correct += (pred_i == true_i and true_i == 1)
        num_neg_correct += (pred_i == true_i and true_i == 0)
    weighted_accuracy = ((weight_pos * num_pos_correct) 
                         + (weight_neg * num_neg_correct))/((weight_pos * num_pos) + (weight_neg * num_neg))
    return weighted_accuracy

#custom scorer based on weighted accuracy function given
def weighted_accuracy_switched(y, y_pred):
    return weighted_accuracy(y_pred, y)

weighted_accuracy_score = make_scorer(weighted_accuracy_switched, greater_is_better = True)

In [10]:
Y = df_w_dup['ipo']
X = df_w_dup[features]

In [10]:
#first try on knn
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3)

tuned_parameters = [{'n_neighbors':[1, 3, 5, 7, 9, 11, 15, 21, 35, 51], 'weights': ['uniform', 'distance']}]

knn_cv = KNeighborsClassifier(n_neighbors = 3)
knn_cv_scores = cross_val_score(knn_cv, X, Y, cv=5, scoring=weighted_accuracy_score)
# knn_cv_scores
print("Accuracy: %0.2f (+/- %0.2f)" % (knn_cv_scores.mean(), knn_cv_scores.std() * 2))

print("# Tuning KNN hyper-parameters for weighted accuracy")
print()

knn_clf = GridSearchCV(KNeighborsClassifier(), tuned_parameters, scoring=weighted_accuracy_score)
knn_clf.fit(X_train, y_train)

print("Best parameters set found on train set:")
print()
print(knn_clf.best_params_)
print()
print("Grid scores on train set:")
print()
means = knn_clf.cv_results_['mean_test_score']
stds = knn_clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, knn_clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full train set.")
print("The scores are computed on the full test set.")
print()
y_true, y_pred = y_test, knn_clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

Accuracy: 0.96 (+/- 0.00)
# Tuning KNN hyper-parameters for weighted accuracy

Best parameters set found on train set:

{'n_neighbors': 9, 'weights': 'uniform'}

Grid scores on train set:

0.984 (+/-0.025) for {'n_neighbors': 1, 'weights': 'uniform'}
0.984 (+/-0.025) for {'n_neighbors': 1, 'weights': 'distance'}
0.993 (+/-0.025) for {'n_neighbors': 3, 'weights': 'uniform'}
0.992 (+/-0.024) for {'n_neighbors': 3, 'weights': 'distance'}
0.994 (+/-0.025) for {'n_neighbors': 5, 'weights': 'uniform'}
0.993 (+/-0.025) for {'n_neighbors': 5, 'weights': 'distance'}
0.994 (+/-0.025) for {'n_neighbors': 7, 'weights': 'uniform'}
0.993 (+/-0.026) for {'n_neighbors': 7, 'weights': 'distance'}
0.994 (+/-0.025) for {'n_neighbors': 9, 'weights': 'uniform'}
0.993 (+/-0.025) for {'n_neighbors': 9, 'weights': 'distance'}
0.994 (+/-0.025) for {'n_neighbors': 11, 'weights': 'uniform'}
0.993 (+/-0.025) for {'n_neighbors': 11, 'weights': 'distance'}
0.994 (+/-0.025) for {'n_neighbors': 15, 'weights': 'unifor

In [11]:
#svm cross validation {'C': 10000.0, 'gamma': 0.05, 'kernel': 'rbf'}
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3)

tuned_parameters = [{'kernel': ['poly', 'rbf', 'linear', 'sigmoid'], 'gamma': [0.1,0.05,0.01], 'C': [1e-2, 1e-1, 1, 10, 100]}]

print("# Tuning SVM hyper-parameters for weighted accuracy")
print()

svm_clf = GridSearchCV(svm.SVC(), tuned_parameters, weighted_accuracy_score)
svm_clf.fit(X_train, y_train)

print("Best parameters set found on train set:")
print()
print(svm_clf.best_params_)
print()
print("Grid scores on train set:")
print()
means = svm_clf.cv_results_['mean_test_score']
stds = svm_clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, svm_clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full train set.")
print("The scores are computed on the full test set.")
print()
y_true, y_pred = y_test, svm_clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

# Tuning SVM hyper-parameters for weighted accuracy

Best parameters set found on train set:

{'C': 100, 'gamma': 0.1, 'kernel': 'linear'}

Grid scores on train set:

0.917 (+/-0.025) for {'C': 0.01, 'gamma': 0.1, 'kernel': 'poly'}
0.917 (+/-0.025) for {'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'}
0.917 (+/-0.025) for {'C': 0.01, 'gamma': 0.1, 'kernel': 'linear'}
0.917 (+/-0.025) for {'C': 0.01, 'gamma': 0.1, 'kernel': 'sigmoid'}
0.917 (+/-0.025) for {'C': 0.01, 'gamma': 0.05, 'kernel': 'poly'}
0.917 (+/-0.025) for {'C': 0.01, 'gamma': 0.05, 'kernel': 'rbf'}
0.917 (+/-0.025) for {'C': 0.01, 'gamma': 0.05, 'kernel': 'linear'}
0.917 (+/-0.025) for {'C': 0.01, 'gamma': 0.05, 'kernel': 'sigmoid'}
0.917 (+/-0.025) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'poly'}
0.917 (+/-0.025) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
0.917 (+/-0.025) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'linear'}
0.917 (+/-0.025) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'sigmoid'}
0.917 (+/-0.025) for {'C': 0.1, '

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.5)

# regressor = RandomForestClassifier(n_estimators=20, random_state=0)
# regressor.fit(X_train, y_train)
# y_pred = regressor.predict(X_test)

# from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


# rfc_cv_score = cross_val_score(regressor, X, Y, cv=10, scoring=weighted_accuracy_score)

# print("=== Confusion Matrix ===")
# print(confusion_matrix(y_test, y_pred))
# print('\n')
# print("=== Classification Report ===")
# print(classification_report(y_test, y_pred))
# print('\n')
# print("=== All AUC Scores ===")
# print(rfc_cv_score)
# print('\n')
# print("=== Mean AUC Score ===")
# print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())


In [13]:
#third try on decision tree
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3)

tuned_parameters = [{'n_estimators': [2, 5, 10, 20]}]

print("# Tuning Random Forest hyper-parameters for weighted accuracy")
print()

rf_clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, scoring=weighted_accuracy_score)
rf_clf.fit(X_train, y_train)

print("Best parameters set found on train set:")
print()
print(rf_clf.best_params_)
print()
print("Grid scores on train set:")
print()
means = rf_clf.cv_results_['mean_test_score']
stds = rf_clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, rf_clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full train set.")
print("The scores are computed on the full test set.")
print()
y_true, y_pred = y_test, rf_clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()


# Tuning Random Forest hyper-parameters for weighted accuracy

Best parameters set found on train set:

{'n_estimators': 10}

Grid scores on train set:

0.969 (+/-0.032) for {'n_estimators': 2}
0.969 (+/-0.030) for {'n_estimators': 5}
0.971 (+/-0.028) for {'n_estimators': 10}
0.971 (+/-0.028) for {'n_estimators': 20}

Detailed classification report:

The model is trained on the full train set.
The scores are computed on the full test set.

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2728
           1       0.75      0.09      0.16        66
          20       1.00      0.97      0.99        68

    accuracy                           0.98      2862
   macro avg       0.91      0.69      0.71      2862
weighted avg       0.97      0.98      0.97      2862




In [14]:
#fourth try on logistic regression with l1
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3)

tuned_parameters = {'C' : np.logspace(-3,3,7), 'penalty':['l1'], 'solver':['saga']}

print("# Tuning Logistic Regression hyper-parameters for weighted accuracy")
print()

logreg_cv = GridSearchCV(LogisticRegression(),tuned_parameters, scoring = weighted_accuracy_score)
logreg_cv.fit(X_train,y_train)

print("Best parameters set found on train set:")
print()
print(logreg_cv.best_params_)
print()
print("Grid scores on train set:")
print()
means = logreg_cv.cv_results_['mean_test_score']
stds = logreg_cv.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, logreg_cv.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full train set.")
print("The scores are computed on the full test set.")
print()
y_true, y_pred = y_test, logreg_cv.predict(X_test)
print(classification_report(y_true, y_pred))
print()


# Tuning Logistic Regression hyper-parameters for weighted accuracy



/Users/dongqing_wang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/dongqing_wang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/dongqing_wang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/dongqing_wang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/dongqing_wang/opt/anaconda3/lib/python3.7/site-packages/sklea

Best parameters set found on train set:

{'C': 0.001, 'penalty': 'l1', 'solver': 'saga'}

Grid scores on train set:

0.953 (+/-0.022) for {'C': 0.001, 'penalty': 'l1', 'solver': 'saga'}
0.953 (+/-0.022) for {'C': 0.01, 'penalty': 'l1', 'solver': 'saga'}
0.953 (+/-0.022) for {'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}
0.953 (+/-0.021) for {'C': 1.0, 'penalty': 'l1', 'solver': 'saga'}
0.953 (+/-0.021) for {'C': 10.0, 'penalty': 'l1', 'solver': 'saga'}
0.953 (+/-0.021) for {'C': 100.0, 'penalty': 'l1', 'solver': 'saga'}
0.953 (+/-0.021) for {'C': 1000.0, 'penalty': 'l1', 'solver': 'saga'}

Detailed classification report:

The model is trained on the full train set.
The scores are computed on the full test set.

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      2720
           1       0.00      0.00      0.00        71
          20       0.00      0.00      0.00        71

    accuracy                           0.95      2862
   macro

/Users/dongqing_wang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/dongqing_wang/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
# test on original data (no duplicates)
# Y = df['ipo']
# X = df[features]
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.3, )

In [12]:
#TODO: insert best parameter here
knn_model = KNeighborsClassifier(n_neighbors = 9)
knn_model.fit(X_train, Y_train)
knn_pred = knn_model.predict(X_val)
    
print("The weighted accuracy for knn is:", weighted_accuracy(knn_pred, Y_val))

The weighted accuracy for knn is: 0.9381130228523197


In [15]:
#TODO: insert best parameter here
svm_model = svm.SVC(C = 100, gamma = 0.1, kernel = 'linear')
svm_model.fit(X_train, Y_train)
svm_pred = svm_model.predict(X_val)

print("The weighted accuracy for svm is:", weighted_accuracy(svm_pred, Y_val))

The weighted accuracy for svm is: 0.9377691438977115


In [13]:
#TODO: insert best parameter here
rf_model =RandomForestClassifier(n_estimators = 10, random_state = 0)
rf_model.fit(X_train, Y_train)
rf_pred = rf_model.predict(X_val)

print("The weighted accuracy for random forest is:", weighted_accuracy(rf_pred, Y_val))

The weighted accuracy for random forest is: 0.9381467268975867


In [14]:
#TODO: insert best parameter here
logit_model = LogisticRegression(penalty = 'l1', solver = 'saga',C = 0.001)
logit_model.fit(X_train, Y_train)
logit_pred = logit_model.predict(X_val)

print("The weighted accuracy for logit is:", weighted_accuracy(logit_pred, Y_val))

The weighted accuracy for logit is: 0.937757909215956
